In [1]:
import numpy as np
import matplotlib.pyplot as plt

from rsbp import RSBP, EntropicRSBP
from rsbp import exact_rsbp, exact_entreg_rsbp, exact_entreg_rsbp_primal
from rsbp import calc_B, calc_f_rsbp, calc_g_rsbp
from sinkhorn_rsbp import calc_U
from sinkhorn_rsbp import robust_ibp_eps
from utils import norm_inf

import time

In [2]:
# Dimension
n = 10
m = 2

# Regularization
tau = np.float64(1.0)

In [3]:
np.random.seed(3698)

# Cost matrix
C = np.random.uniform(low=0.01, high=0.1, size=(m, n, n)).astype(np.float64)
# C = (C + C.T) / 2.0

# Marginal vectors
p = np.random.rand(m, n).astype(np.float64)
p = p / p.sum(1, keepdims=True)

# weight
w = np.random.rand(m).astype(np.float64)
w = w / w.sum()

# Theory checking

Original RSBP problem

In [4]:
rsbp = RSBP(C, p, w, tau)

Entropic regularized RSBP

In [5]:
eps = 0.001

# Entropic regularization parameter
U = calc_U(rsbp, eps)
eta = eps / U

# Convert to Entropic Regularized UOT
ersbp = EntropicRSBP(C, p, w, tau, eta)

Optimal solution for original problem

In [6]:
f_hat, X_hat = exact_rsbp(rsbp)

print('Optimal:', calc_f_rsbp(rsbp, X_hat))

Optimal: 0.019808113478932608


Optimal solution for the dual

In [7]:
_, u_star, v_star = exact_entreg_rsbp(ersbp)
X_star_dual = calc_B(ersbp, u_star, v_star)
print('Optimal:', calc_f_rsbp(rsbp, X_star_dual), calc_g_rsbp(ersbp, X_star_dual))

Optimal: 0.01961633861392187 0.019136928872490716


Optimal solution for the primal **without** the normalization constraint

In [8]:
_, X_star_hat = exact_entreg_rsbp_primal(ersbp)
print(X_star_hat.sum((1, 2)))
print('Optimal:', calc_f_rsbp(rsbp, X_star_hat), calc_g_rsbp(ersbp, X_star_hat))

[0.98038678 0.98038678]
Optimal: 0.019613222270152334 0.01913836533633462


Optimal solution for the primal **with** the normalization constraint

In [9]:
_, X_star = exact_entreg_rsbp_primal(ersbp, with_norm_constraint=True)
print(X_star.sum((1, 2)))
print('Optimal:', calc_f_rsbp(rsbp, X_star), calc_g_rsbp(ersbp, X_star))

[1. 1.]
Optimal: 0.019808113479000775 0.019326755752145312


$$
    X^* = \dfrac{\bar{X^*}}{||\bar{X^*}||_1}
$$

In [10]:
norm_inf(X_star_hat / X_star_hat.sum((1, 2), keepdims=True)  - X_star)

3.9657523888056434e-07

Solve with Sinkhorn iterations

In [11]:
# print(f'Epsilon:', eps)
# start = time.time()

# # Sinkhorn
# _, log = robust_ibp_eps(ersbp, f_hat, eps, 
#                         patience=1000, verbose=True)

# print('Time elapsed:', time.time() - start)

In [12]:
# plt.plot(log['f'][1:])
# plt.axhline(f_optimal, color='red')
# plt.show()

In [13]:
# y = []

# for uk, vk in zip(log['u'], log['v']):
#     Xk = calc_B(ersot, uk, vk)
#     # y.append(norm_inf(Xk - X_hat))
#     y.append(calc_g_rsot(ersot, Xk))

# plt.plot(y)